In [1]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val
%matplotlib inline

In [2]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [3]:
batch_size = 1024
pred_len = 62

In [4]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv').fillna(0)

In [5]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [6]:
X[:,-2*pred_len-430:-2*pred_len,:].shape

(145063, 430, 1)

In [7]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,-2*pred_len-430:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [8]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [9]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=20, save_best_path=save_best_path)


EPOCH 1
Running average loss: 0.306826
VALIDATION LOSS: 0.339932
Elapsed time 128.67716574668884 seconds

EPOCH 2
Running average loss: 0.277324
VALIDATION LOSS: 0.307616
Elapsed time 130.39881134033203 seconds

EPOCH 3
Running average loss: 0.270736
VALIDATION LOSS: 0.334855
Elapsed time 130.2153697013855 seconds

EPOCH 4
Running average loss: 0.271408
VALIDATION LOSS: 0.326930
Elapsed time 130.79535484313965 seconds

EPOCH 5
Running average loss: 0.262628
VALIDATION LOSS: 0.323913
Elapsed time 130.9239535331726 seconds

EPOCH 6
Running average loss: 0.257636
VALIDATION LOSS: 0.293759
Elapsed time 130.9645779132843 seconds

EPOCH 7
Running average loss: 0.260767
VALIDATION LOSS: 0.314939
Elapsed time 131.15691661834717 seconds

EPOCH 8
Running average loss: 0.256758
VALIDATION LOSS: 0.337092
Elapsed time 131.0662636756897 seconds

EPOCH 9
Running average loss: 0.259481
VALIDATION LOSS: 0.313699
Elapsed time 130.8859040737152 seconds

EPOCH 10
Running average loss: 0.249812
VALIDATION

In [12]:
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
model.load_state_dict(torch.load(save_best_path))

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=10, save_best_path=save_best_path)


EPOCH 1
Running average loss: 0.240081
VALIDATION LOSS: 0.277908
Elapsed time 130.3156337738037 seconds

EPOCH 2
Running average loss: 0.234400
VALIDATION LOSS: 0.275602
Elapsed time 131.09065008163452 seconds

EPOCH 3
Running average loss: 0.241276
VALIDATION LOSS: 0.275781
Elapsed time 131.07211685180664 seconds

EPOCH 4
Running average loss: 0.237167
VALIDATION LOSS: 0.275322
Elapsed time 131.07584261894226 seconds

EPOCH 5
Running average loss: 0.242194
VALIDATION LOSS: 0.277560
Elapsed time 131.12250995635986 seconds

EPOCH 6
Running average loss: 0.235518
VALIDATION LOSS: 0.273964
Elapsed time 131.10485434532166 seconds

EPOCH 7
Running average loss: 0.239239
VALIDATION LOSS: 0.275229
Elapsed time 130.93220448493958 seconds

EPOCH 8
Running average loss: 0.240899
VALIDATION LOSS: 0.274285
Elapsed time 131.1715157032013 seconds

EPOCH 9
Running average loss: 0.239816
VALIDATION LOSS: 0.274011
Elapsed time 130.93414759635925 seconds

EPOCH 10
Running average loss: 0.237013
VALIDAT

In [15]:
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
model.load_state_dict(torch.load(save_best_path))

In [16]:
outputs, targets, sequences = model.predict(valloader)

In [17]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [18]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv')
X = train_df.drop('Page', axis=1).values

In [19]:
predictions = scaler.inverse_transform(predictions.T).T
true = X

In [20]:
smapes = val.smape(true[:,-60:], predictions[:,-60:], axis=1)
smapes_clipped = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

../wiki/val.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)
../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator


In [21]:
np.nanmean(smapes), np.nanmean(smapes_clipped)

(45.292513613168069, 43.341507158056743)